# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

┌ Info: Recompiling stale cache file C:\Users\wlkel\.julia\compiled\v1.2\RDatasets\JyIbx.ji for RDatasets [ce6b1742-4840-55fa-b093-852dadbb1d8b]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\wlkel\.julia\compiled\v1.2\MLDataUtils\CQWB9.ji for MLDataUtils [cc2ba9b6-d476-5e6d-8eaf-a92d5412d41d]
└ @ Base loading.jl:1240


## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [3]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [4]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,3.1636,0.0,18.1,0,0.655,5.759,48.2,3.0665,24,666
2,0.05023,35.0,6.06,0,0.4379,5.706,28.4,6.6407,1,304
3,8.20058,0.0,18.1,0,0.713,5.936,80.3,2.7792,24,666
4,0.11329,30.0,4.93,0,0.428,6.897,54.3,6.3361,6,300
5,2.33099,0.0,19.58,0,0.871,5.186,93.8,1.5296,5,403
6,0.05735,0.0,4.49,0,0.449,6.63,56.1,4.4377,3,247
7,0.82526,20.0,3.97,0,0.647,7.327,94.5,2.0788,5,264
8,15.288,0.0,18.1,0,0.671,6.649,93.3,1.3449,24,666
9,1.6566,0.0,19.58,0,0.871,6.122,97.3,1.618,5,403


In [5]:
show(test, allcols=true)

101×14 DataFrame
│ Row │ Crim    │ Zn      │ Indus   │ Chas  │ NOx     │ Rm      │ Age     │
│     │ Float64 │ Float64 │ Float64 │ Int64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ 3.1636  │ 0.0     │ 18.1    │ 0     │ 0.655   │ 5.759   │ 48.2    │
│ 2   │ 0.05023 │ 35.0    │ 6.06    │ 0     │ 0.4379  │ 5.706   │ 28.4    │
│ 3   │ 8.20058 │ 0.0     │ 18.1    │ 0     │ 0.713   │ 5.936   │ 80.3    │
│ 4   │ 0.11329 │ 30.0    │ 4.93    │ 0     │ 0.428   │ 6.897   │ 54.3    │
│ 5   │ 2.33099 │ 0.0     │ 19.58   │ 0     │ 0.871   │ 5.186   │ 93.8    │
│ 6   │ 0.05735 │ 0.0     │ 4.49    │ 0     │ 0.449   │ 6.63    │ 56.1    │
│ 7   │ 0.82526 │ 20.0    │ 3.97    │ 0     │ 0.647   │ 7.327   │ 94.5    │
│ 8   │ 15.288  │ 0.0     │ 18.1    │ 0     │ 0.671   │ 6.649   │ 93.3    │
│ 9   │ 1.6566  │ 0.0     │ 19.58   │ 0     │ 0.871   │ 6.122   │ 97.3    │
│ 10  │ 0.21124 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 5.631   │ 100.0

In [6]:
ols = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                 Estimate  Std. Error    t value  Pr(>|t|)     Lower 95%     Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
(Intercept)   34.4844      5.79849      5.94713     <1e-8    23.0843       45.8845    
Crim          -0.0567733   0.0463356   -1.22526     0.2212   -0.147871      0.0343249 
Zn             0.0412213   0.0153017    2.69391     0.0074    0.0111375     0.0713051 
Indus          0.0620063   0.0698482    0.88773     0.3752   -0.0753187     0.199331  
Chas           1.98135     0.97107      2.04038     0.0420    0.0721758     3.89052   
NOx          -20.9554 

In [7]:
predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
 17.268336447302843
 19.740284608160284
 12.508397212587246
 27.795395871998455
  8.456179553556003
 28.2557731810908  
 34.62763764830688 
 18.196051686196512
 20.281242939309596
 11.16528042212883 
 28.256216882421718
 31.356223274065986
 14.69349195984832 
  ⋮                
 12.49347192314077 
 16.176359438048998
 33.50815138183799 
 22.930133617454352
 21.06553633687571 
 12.197660296148948
 15.449747112466019
 17.19946920518802 
 21.783078693674266
 19.342391307530608
 22.269050985634166
 12.839450311456227

In [8]:
GLM.r²(ols)

0.7523429354011747

In [9]:
GLM.adjr²(ols)

0.7441088130487841